<div dir="rtl">
  <center>
    <img src="https://user-images.githubusercontent.com/7662492/229289746-89c5a4c7-afa6-4d46-a0e6-63dfdeb98285.jpg" width="250px"/>

  <h1>تفريغ - الكتب المُيسّرة</h1>
  </center>

  <p>مرحبًا بك في تفريغ لتفريغ المواد الصوتية والمرئية باستخدام تقنيات الذكاء الاصطناعي. لاستخدام تفريغ:</p>

  <ol>
    <li>قم بإدخال روابط المواد المطلوب تحويلها من منصة YouTube أو أي منصة أخرى في حقل "urls" وتأكد من فصلها بمسافة، أو قم بترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</li>
    <li>(اختياري) قم بتحديد أقل عدد من الكلمات في كل جزء من أجزاء التفريغ. يؤثر هذا في طول الأجزاء التي سيتم تفريغها</li>
    <li>
      إذا كنت تريد استخدام مجموعة نماذج Whisper:
      <ul>
        <li>قم باختيار النموذج المُراد استخدامه في حقل "model"</li>
        <li>قم باختيار لغة المادة في حقل "language"</li>
        <li>قم باختيار المهمة المُراد إنجازها في حقل "task" سواء كانت تفريغ المادة أو ترجمتها للانجليزية</li>
      </ul>
    </li>
    <li>
      إذا كنت تريد استخدام تقنية wit.ai:
      <ul>
        <li>قم بوضع مفتاح wit.ai الخاص بك في حقل "wit_api_key"</li>
        <li>(اختياري) قم بتحديد أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف المخرجات</li>
      </ul>
    </li>
    <li>قم بتشغيل الخلية في الأسفل من خلال الضغط على الدائرة المحتوية على السهم</li>
  </ol>

  <p>يمكنك تجربة التفريغ باستخدام نماذج Whisper وتقنية wit.ai واستخدام التفريغ الأفضل لحالتك. كملاحظة عامة، نماذج Whisper تقوم بتفريغ الهمزات والتشكيلات وعلامات الترقيم بشكل أفضل من wit.ai، لكن wit.ai يُنتج أخطاء إملائية أقل.</p>

  <p>عندما ينتهي التحويل سيتم تنزيل الملفات النصية بشكل تلقائي بصيغة <code>txt</code> و <code>srt</code> وسيكون اسم الملف هو مُعرّف المادة على منصة YouTube الذي يكون في آخر رابط المادة: https://youtu.be/<strong>4h5P7jXvW98</strong>.</p>

  <hr>

  <p>يمكنك متابعة مشروع <strong>الكتب المُيسّرة</strong> والتواصل معنا من خلال:</p>

  <ul>
    <li><a href="https://t.me/ieasybooks">قناتنا على تيليجرام</a></li>
    <li><a href="https://www.youtube.com/@ieasybooks">قناتنا على يوتيوب</a></li>
    <li><a href="https://twitter.com/iieasybooks">حسابنا على تويتر</a></li>
    <li><a href="https://www.facebook.com/ieasybooks">صفحتنا على فيسبوك</a></li>
    <li><a href="https://github.com/ieasybooks">حسابنا على GitHub (للمبرمجين)</a></li>
    <li>بريدنا الالكتروني: easybooksdev@gmail.com</li>
  </ul>
</div>

In [11]:
# @title <h1><center>تفريغ</center></h1>

print('جارٍ تجهيز بيئة العمل.')

# Setup Tafrigh.
!apt-get install -q portaudio19-dev python3-pyaudio > pyaudio_fix_logs.txt
%pip install -U tafrigh[wit,whisper]==1.7.6 > install_logs.txt
%pip install -U yt-dlp["default"] > update_yt_dlp_logs.txt

# Start: Quick fix related to Colab, HuggingFace, and faster-whisper.
!apt install libcublas11 > fix_logs.txt
!pip install ctranslate2==4.5.0 > ctranslate2_downgrade_logs.txt

from huggingface_hub.utils import _runtime
_runtime._is_google_colab = False
# End: Quick fix related to Colab, HuggingFace, and faster-whisper.

# Get inputs.

# @markdown <h1 dir="rtl">مدخلات عامة</h1>

# @markdown <p dir="rtl">روابط المواد المطلوب تفريغها وتأكد من فصلها بمسافة، أو أترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</p>
urls = 'https://www.youtube.com/watch?v=8FDmztkv7hs'  # @param { type: "string" }

# @markdown <p dir="rtl">أقل عدد من الكلمات في كل جزء من أجزاء التفريغ</p>
min_words_per_segment = 1  # @param {type:"slider", min:1, max:100, step:1}

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بنماذج Whisper</h1>

# @markdown <p dir="rtl">النموذج المُراد استخدامه للتفريغ</p>
model = 'large-v3 (أفضل دقة)'  # @param ["large-v3 (أفضل دقة)", "medium", "base", "small", "tiny (أقل دقة)"]

# @markdown <p dir="rtl">(اختياري) لغة المادة</p>
language = 'ar'  # @param ["ar", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]

# @markdown <p dir="rtl">(اختياري) المهمة</p>
task = 'تفريغ'  # @param ["تفريغ", "ترجمة"]

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بتقنية wit.ai</h1>

# @markdown <p dir="rtl">المفتاح الخاص بك على موقع wit.ai</p>
wit_api_key = ''  # @param { type: "string" }

# @markdown <p dir="rtl">(اختياري) أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف SRT</p>
max_cutting_duration = 15  # @param {type:"slider", min:1, max:17, step:1}

if model == 'large-v3 (أفضل دقة)':
    model = 'large-v3'
elif model == 'tiny (أقل دقة)':
    model = 'tiny'

if task == 'تفريغ':
  task = 'transcribe'
elif task == 'ترجمة':
  task = 'translate'

# Imports.
import glob
import os
import re # Added for cleaning filenames
import yt_dlp # Added for fetching video titles
import shutil

from collections import deque

from google.colab import files
from tafrigh import farrigh, Config

# Setup directories.
output_dir = 'output'
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
else:
  # Clean up before start
  shutil.rmtree(output_dir)

# --- Start: Fetch video titles ---
print('جارٍ جلب عناوين الفيديو.')
video_titles = {}
for url in urls.split(' '):
    if url.strip():
        try:
            ydl_opts = {
                'quiet': True, # Suppress yt-dlp output
                'no_warnings': True,
                'extract_flat': True, # Only extract info, don't download
                'force_generic_extractor': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=False)
                video_id = info_dict.get('id')
                video_title = info_dict.get('title')
                if video_id and video_title:
                    video_titles[video_id] = video_title
        except Exception as e:
            print(f"Error fetching info for {url}: {e}")
# --- End: Fetch video titles ---

# Start Tafrigh.
if wit_api_key:
  print('جارٍ تحويل المواد إلى نصوص باستخدام تقنيات wit.ai.')
else:
  print('جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.')

config = Config(
  input=Config.Input(
    urls_or_paths=list(map(str.strip, urls.split(' '))) if len(urls.strip()) else ['.'],
    skip_if_output_exist=False,
    download_retries=3,
    yt_dlp_options='{}',
    verbose=False,
  ),
  whisper=Config.Whisper(
    model_name_or_path=model,
    task=task,
    language=language,
    use_faster_whisper=True,
    beam_size=5,
    ct2_compute_type='default',
  ),
  wit=Config.Wit(
    wit_client_access_tokens=None if len(wit_api_key.strip()) == 0 else wit_api_key.split(),
    max_cutting_duration=max_cutting_duration,
  ),
  output=Config.Output(
    min_words_per_segment=min_words_per_segment,
    save_files_before_compact=False,
    save_yt_dlp_responses=False,
    output_sample=0,
    output_formats=['txt', 'srt'],
    output_dir=output_dir,
  ),
)

deque(farrigh(config), maxlen=0)

# --- Start: Rename files based on video titles ---
print('جارٍ إعادة تسمية الملفات.')

# Get files generated by tafrigh
generated_files = glob.glob(f"{output_dir}/*.txt") + glob.glob(f"{output_dir}/*.srt")
files_to_download = [] # List to store paths of renamed files

for file_path in generated_files:
    file_name = os.path.basename(file_path)
    file_extension = os.path.splitext(file_name)[1]
    video_id_from_file = os.path.splitext(file_name)[0]

    if video_id_from_file in video_titles:
        new_title = video_titles[video_id_from_file]
        # Clean title for filename usage (remove invalid characters)
        # Using a more comprehensive regex to remove characters not suitable for filenames
        new_title_cleaned = re.sub(r'[\\/:*?"<>|.]', '', new_title)
        # Ensure the filename is not empty after cleaning
        if not new_title_cleaned:
            new_title_cleaned = video_id_from_file # Fallback to ID if title becomes empty

        new_file_path = os.path.join(output_dir, f"{new_title_cleaned}{file_extension}")

        # Handle potential filename collisions by appending a counter
        counter = 1
        original_new_file_path = new_file_path
        while os.path.exists(new_file_path):
            new_file_path = os.path.join(output_dir, f"{new_title_cleaned}_{counter}{file_extension}")
            counter += 1

        os.rename(file_path, new_file_path)
        print(f"Renamed '{file_name}' to '{os.path.basename(new_file_path)}'")
        files_to_download.append(new_file_path)
    else:
        # If title not found, keep original name but still add to download list
        files_to_download.append(file_path)
# --- End: Rename files based on video titles ---


# Download all txt and srt files.
print('جارٍ تنزيل الملفات النصية.')

# Remove 'output/archive.txt' if it exists in the list of files to download.
archive_path = os.path.join(output_dir, 'archive.txt')
if archive_path in files_to_download:
    files_to_download.remove(archive_path)

for file in files_to_download:
  files.download(file)


جارٍ تجهيز بيئة العمل.


جارٍ جلب عناوين الفيديو.
جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.


URLs or local paths:   0%|          | 0/1 [00:00<?, ?it/s]WARNING: [youtube] No supported JavaScript runtime could be found. Only deno is enabled by default; to use another runtime add  --js-runtimes RUNTIME[:PATH]  to your command/config. YouTube extraction without a JS runtime has been deprecated, and some formats may be missing. See  https://github.com/yt-dlp/yt-dlp/wiki/EJS  for details on installing one



URL elements:   0%|          | 0/1 [00:00<?, ?it/s]

0.00%|          | 0.00/36.11 [00:00<?, ?sec/s]

8.86%|▉         | 3.20/36.11 [00:04<00:41,  1.27s/sec]

98.26%|█████████▊| 35.48/36.11 [00:05<00:00,  6.34sec/s]

URLs or local paths: 100%|██████████| 1/1 [00:10<00:00, 10.34s/it]

جارٍ إعادة تسمية الملفات.
Renamed '8FDmztkv7hs.txt' to 'أعلام السنة المنشورة - السؤال 208 - الشيخ إبراهيم رفيق.txt'
Renamed '8FDmztkv7hs.srt' to 'أعلام السنة المنشورة - السؤال 208 - الشيخ إبراهيم رفيق.srt'
جارٍ تنزيل الملفات النصية.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>